In [1]:
import os
import sys
import pickle
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append("../")
from useful_functions.check_for_missing_data import check_for_missing_data
from useful_functions.physio_data.pd_dictionary import create_pd_dictionary
from useful_functions.physio_data.preprocess_physio_data import preprocess_physio_data
from useful_functions.physio_data.process_physio_timestamps import process_physio_timestamps

---

In [2]:
driving_data_folder = "../../AdVitam/Exp2/Raw/Driving"
physio_data_folder = "../../AdVitam/Exp2/Raw/Physio/Txt"

In [3]:
drivers_to_exclude = check_for_missing_data(driving_data_folder, physio_data_folder)
drivers_to_exclude.extend(["NST77", "NST11"]) #, "ST22", "NST87", "ST14", "ST12", "NST73", "ST10"])

---

In [4]:
physio_timestamps = pd.read_csv(
    "../../AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)

In [5]:
physio_timestamps = process_physio_timestamps(physio_timestamps, drivers_to_exclude)

In [7]:
physio_timestamps.head()

,subject_id,TriggeredObs1,TakeoverObs1,TriggeredObs2,TakeoverObs2,TriggeredObs3,TakeoverObs3,TriggeredObs4,TakeoverObs4,TriggeredObs5,TakeoverObs5,TriggeredObs6,TakeoverObs6,TOTObs1,TOTObs2,TOTObs3,TOTObs4,TOTObs5,TOTObs6
0,NST01,0 days 00:02:56.705100,0 days 00:03:03.523800,0 days 00:06:56.214000,0 days 00:06:58.690200,0 days 00:08:34.815700,NaT,0 days 00:13:06.640800,NaT,0 days 00:16:23.624000,NaT,0 days 00:18:02.245000,NaT,0 days 00:00:06.818700,0 days 00:00:02.476200,NaT,NaT,NaT,NaT
1,ST02,0 days 00:03:50.756500,0 days 00:03:55.778000,0 days 00:01:49.933400,0 days 00:01:53.351600,0 days 00:10:25.382700,0 days 00:10:29.941600,0 days 00:13:11.720300,0 days 00:13:16.067300,0 days 00:05:57.714400,NaT,0 days 00:07:48.515800,NaT,0 days 00:00:05.021500,0 days 00:00:03.418200,0 days 00:00:04.558900,0 days 00:00:04.347000,NaT,NaT
2,NST03,0 days 00:13:35.204000,0 days 00:13:39.824400,0 days 00:10:19.808800,0 days 00:10:23.032200,0 days 00:04:19.471200,NaT,0 days 00:17:07.400900,NaT,0 days 00:06:18.340900,NaT,0 days 00:18:35.174900,NaT,0 days 00:00:04.620400,0 days 00:00:03.223400,NaT,NaT,NaT,NaT
3,ST04,0 days 00:17:20.361900,0 days 00:17:33.047900,0 days 00:11:25.928100,NaT,0 days 00:04:47.372400,NaT,0 days 00:01:59.926600,NaT,0 days 00:06:50.988200,NaT,0 days 00:14:46.936900,NaT,0 days 00:00:12.686000,NaT,NaT,NaT,NaT,NaT
4,NST05,0 days 00:07:08.961300,0 days 00:07:11.272600,0 days 00:12:23.166400,0 days 00:12:38.273600,0 days 00:14:59.418600,0 days 00:15:02.753600,0 days 00:04:28.071600,0 days 00:04:31.402700,0 days 00:02:23.631400,NaT,0 days 00:10:29.173600,NaT,0 days 00:00:02.311300,0 days 00:00:15.107200,0 days 00:00:03.335000,0 days 00:00:03.331100,NaT,NaT


---

In [6]:
phsyiological_data_dictionary = create_pd_dictionary(physio_data_folder, drivers_to_exclude)

In [ ]:
phsyiological_data_dictionary = preprocess_physio_data(phsyiological_data_dictionary)

In [ ]:
# Find Average Beat/Minute
ST20 = phsyiological_data_dictionary["ST20"]
baseline_data = ST20['baseline']
testing_data = ST20['training']
driving_data = ST20['driving']

In [ ]:
# sliding window
window = 10000  # 10 seconds
step_size = 1000  # 1 second

hrv_data = pd.DataFrame()
for i in range(0, len(baseline_data), step_size):
    if i + window > len(baseline_data):
        break

    # Print progress
    print(f"{i}/{len(baseline_data)} = {i/len(baseline_data)*100}%")
    clear_output(wait=True)

    # Get window data
    window_data = baseline_data[i : i + window]

    # Compute HRV
    window_hrv = nk.hrv(baseline_data, sampling_rate=1000)

    # Add to dataframe
    window_hrv["Time"] = baseline_data["Time"].iloc[i + window]

    # Concatenate with hrv_data
    hrv_data = pd.concat([hrv_data, window_hrv], ignore_index=True)

In [ ]:
ST20 = phsyiological_data_dictionary["ST20"]["driving"]

In [ ]:
# plot the signals
fig, ax = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
ax[0].plot(ST20["ECG_Clean"], label="ECG")
ax[0].set_title("ECG")
ax[1].plot(ST20["EDA_Clean"], label="EDA")
ax[1].set_title("EDA")
ax[2].plot(ST20["RSP_Clean"], label="RSP")
ax[2].set_title("RSP")
plt.show()

---

In [ ]:
os.makedirs("../../AdVitam_Processed/Exp2/Processed/Physio", exist_ok=True)

# Loop through physiological data and save as TXT
for driver in phsyiological_data_dictionary:
    # grab segmented data
    baseline_data = phsyiological_data_dictionary[driver]["baseline"]
    training_data = phsyiological_data_dictionary[driver]["training"]
    driving_data = phsyiological_data_dictionary[driver]["driving"]

    # Create the directory if it does not exist
    os.makedirs("../../AdVitam_Processed/Exp2/Processed/Physio/{driver}", exist_ok=True)

    # save the data as txt
    baseline_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_baseline.txt", index=False)
    training_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_training.txt", index=False)
    driving_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_driving.txt", index=False)


---

# ASIDE:

Participants with issues: ST22, NST87, ST14, ST12, NST73, ST10

---